# Federated PyTorch MNIST Tutorial

In [1]:
#Install dependencies if not already installed
!pip install torch torchvision ..

Processing /home/itrushkin/repos/openfl-fork


  Created wheel for openfl: filename=openfl-1.0.1-py3-none-any.whl size=4252631 sha256=fe29dccb713b5ee8a126f764e9619c0ec6d10a1db1d0bce05e2182c527298391
  Stored in directory: /home/itrushkin/.cache/pip/wheels/ed/ad/82/f26baafab4be32d052fb76a6f56c9a76ce6903cc3eed7bb394
Successfully built openfl
  Attempting uninstall: openfl
    Found existing installation: openfl 1.0.1
    Uninstalling openfl-1.0.1:
      Successfully uninstalled openfl-1.0.1
You should consider upgrading via the '/home/itrushkin/.virtualenvs/openfl_research/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet


After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── plan
│   ├── defaults
│   │   ├── data_loader.yaml
│   │   ├── network.yaml
│   │   ├── tasks_keras.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── task_runner.yaml
│   │   ├── defaults
│   │   ├── tasks_tensorflow.yaml
│   │   ├── aggregator.yaml
│   │   ├── tasks_torch.yaml
│   │   └── tasks_fast_estimator.yaml
│   ├── plan.yaml
│   ├── data.yaml
│   └── cols.yaml
├── logs
├── agg_to_col_two_signed_cert.zip
├── data
│   └── MNIST
│       ├── raw
│       └── processed
├── cert
│   ├── ca
│   │   ├── signing-ca.crt
│   │   ├── root-ca.crt
│   │   ├── signing-ca
│   │   ├── signing-ca.csr
│   │   └── root-ca
│   ├── server
│   │   ├── agg_nnlicv611.inn.intel.com.csr
│   │   ├── agg_nnlicv611.inn.intel.com.key
│   │   └── agg_nnlicv611.inn.intel.com.crt
│   ├── client
│   │   ├── col_two.crt
│   │   ├── col_one.crt
│   │   ├── col_one.key
│   │   └── col_

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [4]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_images,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()
train_labels = one_hot(train_labels,10)

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_images,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:64: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:54: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [5]:
feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.cross_entropy(input=output,target=torch.argmax(target, dim=1))

In [6]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [7]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

In [8]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 30000
Collaborator one's validation data size: 5000

Collaborator two's training data size: 30000
Collaborator two's validation data size: 5000



We can see the current plan values by running the `fx.get_plan()` function

In [9]:
 #Get the current values of the plan. Each of these can be overridden
import json
print(json.dumps(fx.get_plan(), indent=4, sort_keys=True))

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 1,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.opt_treatment": "RESET",
    "collaborator.template": "openfl.component.Collaborator",
    "data_loader.settings.batch_size": 2

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [10]:
def agg_func(tensors, data_sizes):
    return np.median(tensors, axis=0)

In [11]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,
                                   {
                                       'aggregator.settings.rounds_to_train':5,
                                       'tasks.train.aggregation_type': agg_func
                                   })

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:277: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:453: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[0.5092638  0.48201922]


[[[[[ 0.05508939  0.3038397  -0.15677142]
    [ 0.31266254  0.3103413   0.01070586]
    [-0.23311372 -0.02905526  0.32964092]]]


  [[[-0.20531528 -0.04859953 -0.23389588]
    [-0.17773783 -0.32326683 -0.19086799]
    [-0.2723774   0.19368768 -0.07539539]]]


  [[[-0.07317927 -0.23705226 -0.05293163]
    [-0.25901493 -0.05397716 -0.2818169 ]
    [ 0.05445158 -0.0809221  -0.03989086]]]


  [[[-0.08725324 -0.1642036   0.28098962]
    [-0.05006864 -0.24969605  0.25193733]
    [ 0.10399592  0.18565118  0.19510177]]]


  [[[ 0.19113791  0.2871135  -0.10628295]
    [ 0.22252427  0.18861082 -0.14566435]
    [ 0.33799762 -0.1803908   0.12877984]]]


  [[[ 0.23775768 -0.16379051  0.24598661]
    [-0.16855513  0.00096856  0.15354462]
    [-0.29281482  0.06796147  0.27010512]]]


  [[[-0.31956393 -0.2470507   0.12252438]
    [ 0.27900383 -0.12651865 -0.14420539]
    [ 0.3116548   0.08403204  0.20537864]]]


  [[[ 0.23872446  0.11108732 -0.3225457 ]
    [ 0.22773597 -0.14045076 -0.27741787]
    [-

[[7.8871682e-07 5.7357943e-07 1.8322781e-06 1.4789351e-06 1.5489397e-07
  6.5478622e-07 1.4786682e-06 9.7491829e-07 7.6849329e-07 5.1625977e-07
  9.7036548e-07 1.4632564e-06 5.4890859e-07 1.2929435e-06 1.1026518e-06
  6.1041288e-07 6.7470978e-07 5.2626098e-07 1.0822280e-06 1.0056079e-06
  8.3605249e-07 6.5220667e-07 1.0270927e-06 7.3098681e-07 8.0492777e-07
  6.2886176e-07 7.8775673e-07 6.1632727e-07 6.7934553e-07 8.3091044e-07
  8.2824022e-07 4.8157915e-07]
 [5.6246307e-07 6.5081650e-07 1.5200515e-06 9.0950681e-07 2.5539430e-07
  6.6673857e-07 1.6168153e-06 9.2673969e-07 9.4645287e-07 6.0294889e-07
  9.4141274e-07 8.0933614e-07 3.8079060e-07 1.2711051e-06 1.1653485e-06
  5.5507098e-07 5.4213137e-07 5.7649044e-07 9.4675704e-07 9.9473402e-07
  8.1394217e-07 4.9290855e-07 1.1154278e-06 7.5392734e-07 6.8082903e-07
  6.3998772e-07 9.4552809e-07 5.8934779e-07 5.8977650e-07 8.5586004e-07
  8.6790823e-07 5.0211412e-07]]
[[938.]
 [938.]]
[[[ 3.1767209e-04  4.5695502e-05  2.7456004e-03 ...  1.1

...

[0.06514921 0.06826576]


[[[[[ 0.05730506  0.30770686 -0.15992267]
    [ 0.31842792  0.31939965  0.01722945]
    [-0.23548852 -0.02404473  0.32777685]]]


  [[[-0.19095527 -0.03594233 -0.22296064]
    [-0.1778941  -0.29089728 -0.18586223]
    [-0.24320899  0.23040733 -0.05795427]]]


  [[[-0.0712956  -0.22346471 -0.02998039]
    [-0.23064479 -0.03537453 -0.28135335]
    [ 0.09304883 -0.06229437 -0.03881381]]]


  [[[-0.1080092  -0.17251417  0.2804981 ]
    [-0.0627408  -0.25108325  0.25717133]
    [ 0.09141191  0.17726019  0.19422948]]]


  [[[ 0.18451956  0.28467074 -0.10483456]
    [ 0.21969168  0.19093259 -0.14664821]
    [ 0.3369443  -0.1785183   0.12237554]]]


  [[[ 0.22599941 -0.1542556   0.24822934]
    [-0.18177211  0.00442567  0.15826514]
    [-0.31071293  0.05866947  0.2697056 ]]]


  [[[-0.33370787 -0.26255503  0.10248559]
    [ 0.26906744 -0.1296628  -0.15590683]
    [ 0.30863068  0.09078141  0.20230868]]]


  [[[ 0.2360678   0.10072105 -0.3400973 ]
    [ 0.23776196 -0.12744698 -0.27771163]
    [-

[[[[[8.07516836e-03 6.04276685e-03 6.27330225e-03]
    [9.37913358e-03 8.94849841e-03 8.27789027e-03]
    [5.63887786e-03 6.56201411e-03 8.57316982e-03]]

   [[7.10034146e-06 5.42422231e-06 4.78069660e-06]
    [8.72173700e-07 9.65846766e-07 1.79436097e-06]
    [2.12583288e-07 1.62704836e-07 3.44437893e-07]]

   [[2.00423733e-06 5.05241997e-06 6.80401308e-06]
    [1.26600881e-07 5.11915516e-07 1.97179520e-06]
    [6.07711456e-07 3.47164672e-07 1.44823673e-07]]

   ...

   [[7.43497920e-04 7.29445135e-04 6.43041451e-04]
    [5.83377085e-04 6.67702116e-04 8.06928554e-04]
    [3.94391565e-04 5.09928446e-04 4.31917753e-04]]

   [[1.09715702e-03 9.46533866e-04 1.18762837e-03]
    [2.06626719e-03 1.71606720e-03 1.55341509e-03]
    [1.58364780e-03 1.57793437e-03 1.68593053e-03]]

   [[6.01950778e-05 6.79352306e-05 6.60308942e-05]
    [3.63858635e-05 7.70828483e-05 3.93165392e-05]
    [4.53056164e-05 5.00003953e-05 5.18337401e-05]]]


  [[[9.14504193e-03 1.52263036e-02 1.59412101e-02]
    [1.32

[[2.4108127e-07 6.7255365e-07 9.5749465e-07 3.0594308e-07 7.3666558e-07
  1.2614234e-06 6.2880400e-07 3.7190958e-07 3.8009637e-07 8.6811463e-07
  5.6298268e-07 6.5572448e-07 2.5599516e-07 9.6734721e-07 3.9030056e-07
  2.5795717e-07 7.3258411e-07 5.0221581e-07 5.6900939e-07 3.6527587e-07
  3.1102454e-07 5.2874441e-07 7.5379756e-07 5.5459253e-07 8.1145544e-07
  3.4888546e-07 3.1196959e-07 4.0902029e-07 3.1433959e-07 5.1102541e-07
  5.0106024e-07 1.1412772e-06]
 [2.5010837e-07 6.5185986e-07 9.8742885e-07 3.1820562e-07 6.5953094e-07
  1.2750792e-06 6.2589601e-07 3.9460912e-07 3.9483527e-07 8.3401113e-07
  6.2538294e-07 6.1083358e-07 2.5070617e-07 9.1949238e-07 4.4038748e-07
  2.3732375e-07 7.3712687e-07 4.7854974e-07 6.8801239e-07 3.5759169e-07
  3.0584457e-07 5.2721794e-07 7.9226800e-07 5.2125046e-07 8.7592844e-07
  3.2414528e-07 3.1665249e-07 4.1303392e-07 3.1319982e-07 5.6562504e-07
  5.0854771e-07 1.1411571e-06]]
[[4690.]
 [4690.]]
[[[ 7.3996029e-07  5.1835930e-04  1.0189334e-03 ... -8

[[4690.]
 [4690.]]
[[-4.0848632e-03 -4.2811516e-04  4.7489672e-04  5.2675820e-04
   1.0106217e-03  6.9172739e-04  4.2297761e-03 -1.7057484e-03
  -5.1159627e-04 -2.0346213e-04]
 [ 8.1323400e-05 -7.1820494e-04  1.8740875e-03 -3.9448743e-03
   1.8609125e-03  5.1514269e-04  7.7602075e-04 -1.7314232e-03
   2.3590822e-03 -1.0720687e-03]]
[[6.7811539e-05 7.0177084e-05 1.3945648e-04 1.5393175e-04 1.0915299e-04
  1.3651601e-04 8.5066102e-05 1.4115413e-04 2.0782962e-04 1.7818090e-04]
 [8.8793713e-05 7.3492149e-05 1.3444672e-04 1.4215309e-04 1.2956756e-04
  1.3137267e-04 9.8142322e-05 1.3512500e-04 1.9423064e-04 1.9385859e-04]]
[[10.]
 [10.]]


In [12]:
#Save final model
final_fl_model.save_native('final_pytorch_model')

In [13]:
import pickle
print(pickle.dumps(1))

b'\x80\x04K\x01.'
